# Premade Estimators

[official tutorial](https://www.tensorflow.org/tutorials/estimator/premade)

This tutorial shows you how to solve the Iris classification problem in TensorFlow using Estimators. An Estimator is TensorFlow's high-level representation of a complete model, and it has been designed for easy scaling and asynchronous training. For more details see [Estimators](https://www.tensorflow.org/guide/estimator).

Note that in TensorFlow 2.0, the Keras API can accomplish many of these same tasks, and is believed to be an easier API to learn. If you are starting fresh, we would recommend you start with Keras. For more information about the available high level APIs in TensorFlow 2.0, see Standardizing on Keras.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [4]:
train.shape, train_y.shape

((120, 4), (120,))

## Overview of programming with Estimators

Now that you have the data set up, you can define a model using a TensorFlow Estimator. An Estimator is any class derived from [tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator). TensorFlow provides a collection of [tf.estimator](https://www.tensorflow.org/api_docs/python/tf/estimator) (for example, `LinearRegressor`) to implement common ML algorithms. Beyond those, you may write your own [custom Estimators](https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator). We recommend using pre-made Estimators when just getting started.

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

* Create one or more input functions.
* Define the model's feature columns.
* Instantiate an Estimator, specifying the feature columns and various hyperparameters.
* Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data.

Let's see how those tasks are implemented for Iris classification.

## Create input functions

You must create input functions to supply data for training, evaluating, and prediction.

An **input function** is a function that returns a `tf.data.Dataset` object which outputs the following two-element tuple:

* [`features`](https://developers.google.com/machine-learning/glossary/#feature) - A Python dictionary in which:
    * Each key is the name of a feature.
    * Each value is an array containing all of that feature's values.
* `label` - An array containing the values of the [label](https://developers.google.com/machine-learning/glossary/#label) for every example.

Your input function may generate the features dictionary and label list any way you like. However, we recommend using TensorFlow's [Dataset API](https://www.tensorflow.org/guide/data), which can parse all sorts of data.

The Dataset API can handle a lot of common cases for you. For example, using the Dataset API, you can easily read in records from a large collection of files in parallel and join them into a single stream.

To keep things simple in this example you are going to load the data with [pandas](https://pandas.pydata.org/), and build an input pipeline from this in-memory data:

In [5]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [7]:
input_fn_iterator = input_fn(features=train, labels=train_y, batch_size=5)
input_fn_iterator

<DatasetV1Adapter shapes: ({SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, (?,)), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int32)>

In [13]:
next(iter(input_fn_iterator))

({'SepalLength': <tf.Tensor: id=33, shape=(5,), dtype=float64, numpy=array([7.7, 6.6, 6.7, 6.5, 5.2])>,
  'SepalWidth': <tf.Tensor: id=34, shape=(5,), dtype=float64, numpy=array([2.6, 2.9, 3. , 3. , 2.7])>,
  'PetalLength': <tf.Tensor: id=31, shape=(5,), dtype=float64, numpy=array([6.9, 4.6, 5.2, 5.5, 3.9])>,
  'PetalWidth': <tf.Tensor: id=32, shape=(5,), dtype=float64, numpy=array([2.3, 1.3, 2.3, 1.8, 1.4])>},
 <tf.Tensor: id=35, shape=(5,), dtype=int32, numpy=array([2, 1, 2, 2, 1], dtype=int32)>)

## Define the feature columns

A [feature column](https://developers.google.com/machine-learning/glossary/#feature_columns) is an object describing how the model should use raw input data from the features dictionary. When you build an Estimator model, you pass it a list of [ `feature columns` ](https://www.tensorflow.org/api_docs/python/tf/feature_column) that describes each of the features you want the model to use. [`The tf.feature_column`](https://www.tensorflow.org/api_docs/python/tf/feature_column) module provides many options for representing data to the model.

For Iris, the 4 raw features are numeric values, so we'll build a list of feature columns to tell the Estimator model to represent each of the four features as 32-bit floating-point values. Therefore, the code to create the feature column is:

In [10]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Feature columns can be far more sophisticated than those we're showing here. You can read more about Feature Columns in [this guide](https://www.tensorflow.org/tutorials/structured_data/feature_columns).

Now that you have the description of how you want the model to represent the raw features, you can build the estimator

### Numeric columns  
The output of a feature column becomes the input to the model (using the demo function defined above, we will be able to see exactly how each column from the dataframe is transformed). A [numeric column](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

In [28]:
feat_dict_batch = next(iter(input_fn_iterator))[0]
feat_dict_batch

{'SepalLength': <tf.Tensor: id=247, shape=(5,), dtype=float64, numpy=array([7.7, 6.6, 6.7, 6.5, 5.2])>,
 'SepalWidth': <tf.Tensor: id=248, shape=(5,), dtype=float64, numpy=array([2.6, 2.9, 3. , 3. , 2.7])>,
 'PetalLength': <tf.Tensor: id=245, shape=(5,), dtype=float64, numpy=array([6.9, 4.6, 5.2, 5.5, 3.9])>,
 'PetalWidth': <tf.Tensor: id=246, shape=(5,), dtype=float64, numpy=array([2.3, 1.3, 2.3, 1.8, 1.4])>}

In [29]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(feat_dict_batch).numpy())

In [30]:
demo(my_feature_columns[0]) # fist column

[[7.7]
 [6.6]
 [6.7]
 [6.5]
 [5.2]]


In [31]:
demo(my_feature_columns) # all columns 

[[6.9 2.3 7.7 2.6]
 [4.6 1.3 6.6 2.9]
 [5.2 2.3 6.7 3. ]
 [5.5 1.8 6.5 3. ]
 [3.9 1.4 5.2 2.7]]


## Instantiate an estimator
The Iris problem is a classic classification problem. Fortunately, TensorFlow provides several pre-made classifier Estimators, including:

* `tf.estimator.DNNClassifier` for deep models that perform multi-class classification.
* `tf.estimator.DNNLinearCombinedClassifier` for wide & deep models.
* `tf.estimator.LinearClassifier` for classifiers based on linear models.

For the Iris problem, `tf.estimator.DNNClassifier` seems like the best choice. Here's how you instantiated this Estimator:

In [32]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpz8s3nhzg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feff2f13450>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate, and Predict

Now that you have an Estimator object, you can call methods to do the following:

* Train the model.
* Evaluate the trained model.
* Use the trained model to make predictions.

### Train the model
Train the model by calling the Estimator's train method as follows:

In [33]:
# Train the Model.
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True),steps=500)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpz8s3nhzg/model.ckpt.
INFO:tensorflow:loss = 391.49762, step = 0
INFO:tensorflow:global_step/sec: 385.529
INFO:tensorflow:loss = 37.06316, step = 100 (0.260 sec)
INFO:tensorflow:global_step/sec: 491.984
INFO:tensorfl

Note that you wrap up your input_fn call in a `lambda` to capture the arguments while providing an input function that takes no arguments, as expected by the Estimator.   
The steps argument tells the method to stop training after a number of training steps.

### Evaluate the trained model
Now that the model has been trained, you can get some statistics on its performance.   
The following code block evaluates the accuracy of the trained model on the test data:

In [35]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-21T09:49:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpz8s3nhzg/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-01-21-09:49:14
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.96666664, average_loss = 0.069623835, global_step = 500, loss = 2.088715
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: /tmp/tmpz8s3nhzg/model.ckpt-500

Test set accuracy: 0.967



In [36]:
eval_result

{'accuracy': 0.96666664,
 'average_loss': 0.069623835,
 'loss': 2.088715,
 'global_step': 500}

Unlike the call to the train method, you did not pass the `steps` argument to evaluate. The `input_fn` for eval only yields a single `epoch` of data.

The `eval_result` dictionary also contains the `average_loss` (mean loss per sample), the `loss` (mean loss per mini-batch) and the value of the estimator's `global_step` (the number of training iterations it underwent).

### Making predictions (inferring) from the trained model
You now have a trained model that produces good evaluation results. You can now use the trained model to predict the species of an Iris flower based on some unlabeled measurements. As with training and evaluation, you make predictions using a single function call:

In [44]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [45]:
pred_iterater = input_fn(predict_x)
pred_iterater

<DatasetV1Adapter shapes: {SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, types: {SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}>

In [59]:
next(iter(pred_iterater))

{'SepalLength': <tf.Tensor: id=2995, shape=(3,), dtype=float32, numpy=array([5.1, 5.9, 6.9], dtype=float32)>,
 'SepalWidth': <tf.Tensor: id=2996, shape=(3,), dtype=float32, numpy=array([3.3, 3. , 3.1], dtype=float32)>,
 'PetalLength': <tf.Tensor: id=2993, shape=(3,), dtype=float32, numpy=array([1.7, 4.2, 5.4], dtype=float32)>,
 'PetalWidth': <tf.Tensor: id=2994, shape=(3,), dtype=float32, numpy=array([0.5, 1.5, 2.1], dtype=float32)>}

In [64]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))
predictions

<generator object Estimator.predict at 0x7fef64264c50>

The `predict` method returns a Python iterable, yielding a dictionary of prediction results for each example. The following code prints a few predictions and their probabilities:

In [65]:
next(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpz8s3nhzg/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([ 11.1057005,   5.4014263, -13.83364  ], dtype=float32),
 'probabilities': array([9.9667943e-01, 3.3206313e-03, 1.4707452e-11], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1, 2], dtype=int32),
 'all_classes': array([b'0', b'1', b'2'], dtype=object)}

In [58]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpz8s3nhzg/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (99.7%), expected "Setosa"
Prediction is "Versicolor" (99.6%), expected "Versicolor"
Prediction is "Virginica" (96.0%), expected "Virginica"
